# PROBLEM SET 2

In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('/Users/jugalmarfatia/Documents/fall 2018/econometrics_2/matching_estimators/matching_estimators/bwght2.csv')

## Drop null values and display first 5 rows to check dataset.
 

In [2]:
df = df.dropna()
df.head()

,mage,meduc,monpre,npvis,fage,feduc,bwght,omaps,fmaps,cigs,...,male,mwhte,mblck,moth,fwhte,fblck,foth,lbwght,magesq,npvissq
0,26,12.0,2.0,12.0,34.0,16.0,3060,9.0,9.0,0.0,...,1,0,0,1,0,0,1,8.026170,676,144.0
2,33,12.0,1.0,12.0,36.0,16.0,2530,8.0,9.0,0.0,...,0,1,0,0,1,0,0,7.835975,1089,144.0
3,28,17.0,5.0,8.0,32.0,17.0,3289,8.0,9.0,0.0,...,1,1,0,0,1,0,0,8.098339,784,64.0
4,23,13.0,2.0,6.0,24.0,16.0,3590,6.0,8.0,0.0,...,1,1,0,0,1,0,0,8.185907,529,36.0
5,28,12.0,1.0,12.0,30.0,16.0,3420,9.0,9.0,0.0,...,0,1,0,0,1,0,0,8.137396,784,144.0


## Create dummy variable for ever smoked

In [3]:
df['cigs_dummy'] = (df['cigs'] > 0).astype(int)

## Difference in log birthweight of smokers vs non-smokers

In [4]:
s_lbwght, ns_lbwght = df.groupby('cigs_dummy')['lbwght'].mean()

diff = (s_lbwght - ns_lbwght)

In [5]:
print "Difference in log birthweight = " + str(diff)

Difference in log birthweight = 0.0629659722452


## Run OLS regression 

In [6]:
import statsmodels.formula.api as sm
ols = sm.ols(formula="lbwght ~ cigs_dummy + mage + meduc + monpre + npvis + fage + feduc + fblck + magesq + npvissq + mblck", 
             data=df).fit()


## Results from the regression. 

In [7]:
print ols.summary()


                            OLS Regression Results                            
Dep. Variable:                 lbwght   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     4.459
Date:                Thu, 06 Sep 2018   Prob (F-statistic):           1.19e-06
Time:                        09:44:42   Log-Likelihood:                 445.31
No. Observations:                1612   AIC:                            -866.6
Df Residuals:                    1600   BIC:                            -802.0
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      7.5602      0.140     53.910      0.0

## Predict propensity score

In [8]:
logit = sm.logit(formula="cigs_dummy ~ mage + meduc + monpre + npvis + fage + feduc + fblck + magesq + npvissq + mblck", 
                data=df).fit()
print logit.summary()

df['propensity_score'] = logit.predict()

Optimization terminated successfully.
         Current function value: 0.268796
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:             cigs_dummy   No. Observations:                 1612
Model:                          Logit   Df Residuals:                     1601
Method:                           MLE   Df Model:                           10
Date:                Thu, 06 Sep 2018   Pseudo R-squ.:                 0.08027
Time:                        09:44:42   Log-Likelihood:                -433.30
converged:                       True   LL-Null:                       -471.12
                                        LLR p-value:                 3.575e-12
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      7.1956      2.349      3.064      0.002         2.592    11.799
mage          -0.2917      0.

## Function to match

In [9]:
def find_match(value, array):
    dist = np.array(abs(array - value))
    return dist.argmin()

def match(original, matching, propensity_var, outcome_var):
    original["m_" + outcome_var] = 0
    original["m_" + propensity_var + "dis"] = 0

    matching_array = matching[propensity_var]
    for i in range(len(original)):
        pair = find_match(original.loc[i, propensity_var], matching_array)
        original.loc[i,"m_" + outcome_var] = matching.loc[pair, outcome_var]
        original.loc[i,"m_" + propensity_var + "dis" ] = original.loc[i,propensity_var] - matching_array[pair]

    return original 
        

### In this the match is accomplished by finding the nearest propensity score

## Match 1:1 

In [10]:
treated = df.loc[df.cigs_dummy == 1].reset_index()
control = df.loc[df.cigs_dummy == 0].reset_index()

treated = match(treated, control, 'propensity_score', 'lbwght')
control = match(control, treated, 'propensity_score', 'lbwght')

## Function to estimate treatment effect

In [11]:
def avg_treatment_effect(treated, control, outcome_var):
    treated['diff'] = treated[outcome_var] - treated['m_' + outcome_var]
    control['diff'] = control['m_' + outcome_var] - control[outcome_var] 

    df = treated.append(control, ignore_index=True)
    return df['diff'].mean()

def avg_treatment_effect_on_treated(treated,  outcome_var):
    df = treated
    df['diff'] = df[outcome_var] - df['m_' + outcome_var]
    return df['diff'].mean()

## Display Average Treatment effect

In [12]:
print avg_treatment_effect(treated, control, 'lbwght')

-0.0445944706805


## Display Average Treatment effect on the Treated

In [13]:
print avg_treatment_effect_on_treated(treated, 'lbwght')

-0.0428045313768


## Plot the distribution of the propensity score of treated vs non-treat

In [14]:
import plotly.plotly as py
import plotly.figure_factory as ff

hist_data = [treated['propensity_score'].tolist(), control['propensity_score'].tolist()]

group_labels = ['Treated', 'Control']

fig = ff.create_distplot(hist_data, group_labels, bin_size=.1)

py.iplot(fig, filename='Distplot with Multiple Datasets')

### Since the mean and distribution of the two groups are significantly the matching procedure conducted above do not give conclusive estimates. 